In [1]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils import data

# Libraries for dataset
import torchvision
from torchvision import transforms, datasets

# Miscellaneous Libraries
import time

# For DataLoader
from torch.utils.data import Dataset

# For Dataset Files
import os

import PIL
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# Define Hyperparameters

num_epochs = 24

batch = 16

learning_rate = 0.01

momentum = 0.5

In [3]:
if os.path.exists(os.getcwd() + "/ASL_Sorted_List/"):
    print("This is either sorted correctly or there is a mistake." 
         + "\nIf there is a mistake, delete the /ASL_Sorted_List"
         + "\nThen run this cell again.")
else:
    (_, _, files) = next(os.walk(os.getcwd() + '/ASL_Combined_Dataset'))
    (_, train_or_test, _) = next(os.walk(os.getcwd() + '/Combined_Dataset2'))
    
    if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/"):
        os.mkdir(os.getcwd() + "/ASL_Sorted_List/")
        if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/Training_Data/"):
            os.mkdir(os.getcwd() + "/ASL_Sorted_List/Training_Data/")
        if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/Testing_Data/"):
            os.mkdir(os.getcwd() + "/ASL_Sorted_List/Testing_Data/")
    a = set()
    for file in files:
        if file[4] == '5':
            img = Image.open(os.getcwd() + '/ASL_Combined_Dataset/' + file)
            name = file
            name = name.split('_')
            label = name[1]
            if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/Testing_Data/" + label + "/"):
                os.mkdir(os.getcwd() + "/ASL_Sorted_List/Testing_Data/" + label + "/")
            img.save(os.getcwd() +'/ASL_Sorted_List/Testing_Data/' + label + "/" + file)
        else:
            img = Image.open(os.getcwd() + '/ASL_Combined_Dataset/' + file)
            name = file
            name = name.split('_')
            label = name[1]
            if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/Training_Data/" + label + "/"):
                os.mkdir(os.getcwd() + "/ASL_Sorted_List/Training_Data/" + label + "/")
            img.save(os.getcwd() + '/ASL_Sorted_List/Training_Data/' + label + "/"+ file)
            
    for setName in train_or_test:
        (_, labs, _) = next(os.walk(os.getcwd() + '/Combined_Dataset2/' + str(setName)))
        for labels2 in labs:
            (_, _, files) = next(os.walk(os.getcwd() 
                                           + '/Combined_Dataset2/' 
                                           + str(setName) + '/' + str(labels2)))
            for file2 in files:
                img = Image.open(os.getcwd() + '/Combined_Dataset2/' + str(setName) + "/"
                                 + str(labels2) + "/" + str(file2))
                if not os.path.exists(os.getcwd() + "/ASL_Sorted_List/" + str(setName) 
                                      + "/" + labels2 + "/"):
                    os.mkdir(os.getcwd() + "/ASL_Sorted_List/" + str(setName) + "/" + labels2 + "/")
                img.save(os.getcwd() + "/ASL_Sorted_List/" + str(setName) + "/" + labels2 + "/" + file2)
        

This is either sorted correctly or there is a mistake.
If there is a mistake, delete the /ASL_Sorted_List
Then run this cell again.


In [4]:
data_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
testing_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.223, 0.225])
])

training_dataset = datasets.ImageFolder(root='ASL_Sorted_List/Training_Data/',
                                           transform=data_transform)
training_loader = torch.utils.data.DataLoader(training_dataset,
                                             batch_size=batch, shuffle=True)

testing_dataset = datasets.ImageFolder(root='ASL_Sorted_List/Testing_Data/',
                                           transform=testing_transform)
testing_loader = torch.utils.data.DataLoader(testing_dataset,
                                            batch_size=batch, shuffle=False)

In [5]:
# to_pil = torchvision.transforms.ToPILImage()

# for sample in training_loader:
#     print('Image as a tensor:')
#     print(sample[0])
    
#     print('\nClassification:')
#     print(sample[1])
#     break

In [6]:
if os.path.exists("Saved_Model/aslPaper1Model.pth"):
    model = torch.hub.load('pytorch/vision:v0.4.2', 'googlenet', pretrained=True)
    model.eval()
    print("Model Loading From SaveState")
    model.load_state_dict(torch.load("Saved_Model/aslPaper1Model.pth"))
else:
    model = torch.hub.load('pytorch/vision:v0.4.2', 'googlenet', pretrained=True)
    model.eval()

# Cross Entropy Loss Function
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (SGD)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)  
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Set model to training (updating weights)
model.train()

Using cache found in /home/yvillia/.cache/torch/hub/pytorch_vision_v0.4.2


Model Loading From SaveState


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [7]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(training_loader):
        if(Y_train_batch.shape[0]<batch):
            continue
        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           
        
        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(training_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')
    
    
# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

#Save Model For Future Usage
print("Saved")
torch.save(model.state_dict(), "Saved_Model/aslPaper1Model.pth")

Epoch: 1/24, Step: 1/1034, Loss: 0.0032780487090349197, Accuracy: 100.0%
Epoch: 1/24, Step: 101/1034, Loss: 0.024393673986196518, Accuracy: 100.0%
Epoch: 1/24, Step: 201/1034, Loss: 0.0015803457936272025, Accuracy: 100.0%
Epoch: 1/24, Step: 301/1034, Loss: 0.0006119731697253883, Accuracy: 100.0%
Epoch: 1/24, Step: 401/1034, Loss: 0.00022071629064157605, Accuracy: 100.0%
Epoch: 1/24, Step: 501/1034, Loss: 0.003861022414639592, Accuracy: 100.0%
Epoch: 1/24, Step: 601/1034, Loss: 0.007250823546200991, Accuracy: 100.0%
Epoch: 1/24, Step: 701/1034, Loss: 0.0012948379153385758, Accuracy: 100.0%
Epoch: 1/24, Step: 801/1034, Loss: 0.0020189667120575905, Accuracy: 100.0%
Epoch: 1/24, Step: 901/1034, Loss: 0.12393272668123245, Accuracy: 93.75%
Epoch: 1/24, Step: 1001/1034, Loss: 0.013757627457380295, Accuracy: 100.0%
Epoch: 2/24, Step: 1/1034, Loss: 0.0009698880603536963, Accuracy: 100.0%
Epoch: 2/24, Step: 101/1034, Loss: 0.0016858569579198956, Accuracy: 100.0%
Epoch: 2/24, Step: 201/1034, Loss

Epoch: 11/24, Step: 1/1034, Loss: 0.00025595410261303186, Accuracy: 100.0%
Epoch: 11/24, Step: 101/1034, Loss: 0.004119712393730879, Accuracy: 100.0%
Epoch: 11/24, Step: 201/1034, Loss: 0.00045016591320745647, Accuracy: 100.0%
Epoch: 11/24, Step: 301/1034, Loss: 0.01219236757606268, Accuracy: 100.0%
Epoch: 11/24, Step: 401/1034, Loss: 5.725724622607231e-05, Accuracy: 100.0%
Epoch: 11/24, Step: 501/1034, Loss: 0.00021048096823506057, Accuracy: 100.0%
Epoch: 11/24, Step: 601/1034, Loss: 0.0022796383127570152, Accuracy: 100.0%
Epoch: 11/24, Step: 701/1034, Loss: 0.00014042362454347312, Accuracy: 100.0%
Epoch: 11/24, Step: 801/1034, Loss: 1.652473110880237e-05, Accuracy: 100.0%
Epoch: 11/24, Step: 901/1034, Loss: 4.456468013813719e-05, Accuracy: 100.0%
Epoch: 11/24, Step: 1001/1034, Loss: 0.0006554264109581709, Accuracy: 100.0%
Epoch: 12/24, Step: 1/1034, Loss: 0.00017658376600593328, Accuracy: 100.0%
Epoch: 12/24, Step: 101/1034, Loss: 0.0006051238742657006, Accuracy: 100.0%
Epoch: 12/24,

Epoch: 20/24, Step: 1001/1034, Loss: 3.041888339794241e-05, Accuracy: 100.0%
Epoch: 21/24, Step: 1/1034, Loss: 0.00022206227004062384, Accuracy: 100.0%
Epoch: 21/24, Step: 101/1034, Loss: 0.00760152330622077, Accuracy: 100.0%
Epoch: 21/24, Step: 201/1034, Loss: 0.0013772790553048253, Accuracy: 100.0%
Epoch: 21/24, Step: 301/1034, Loss: 0.00010337174899177626, Accuracy: 100.0%
Epoch: 21/24, Step: 401/1034, Loss: 0.008871113881468773, Accuracy: 100.0%
Epoch: 21/24, Step: 501/1034, Loss: 0.00012565958604682237, Accuracy: 100.0%
Epoch: 21/24, Step: 601/1034, Loss: 0.0008017398649826646, Accuracy: 100.0%
Epoch: 21/24, Step: 701/1034, Loss: 0.000266972288955003, Accuracy: 100.0%
Epoch: 21/24, Step: 801/1034, Loss: 0.0001447486283723265, Accuracy: 100.0%
Epoch: 21/24, Step: 901/1034, Loss: 0.0001504369720350951, Accuracy: 100.0%
Epoch: 21/24, Step: 1001/1034, Loss: 0.0002890562464017421, Accuracy: 100.0%
Epoch: 22/24, Step: 1/1034, Loss: 0.0003244567778892815, Accuracy: 100.0%
Epoch: 22/24, S

In [8]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in testing_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 93.60321384425217%
